In [1]:
import numpy as np
import scipy as sp
from scipy import signal
import scipy.io as io
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from os import path

In [2]:
# generamos un electrodo promedio por cada persona 
# (promediando los 5 electrodos del enunciado que se encuentran juntos)
# y los guardamos en disco. De esta manera a partir de ahora vamos a trabajar solo con esos archivos que son 
# mucho más livianos y tiene toda la información que necesitamos, 894 epochs por cada paciente.

electrodos_a_promediar = [7, 43, 79, 130, 184]

for t in ['P', 'S']:
    for i in range(1,11): 
        try:
            data_file = path.join('../../../../data/',t) + "%02d" % (i) + '.mat'
            elec_prom_file = 'elec_prom_' + t + "%02d" % (i) + '.dmp'
            data = io.loadmat(data_file)['data']
            electrodo_promedio = data[:,electrodos_a_promediar,:].mean(axis=1)
            electrodo_promedio.dump(elec_prom_file)
        except Exception as exception:
            print(str(exception))
            pass

(894, 256, 201)
(894, 201)
(687, 256, 201)
(687, 201)
(700, 256, 201)
(700, 201)
(1596, 256, 201)
(1596, 201)
(857, 256, 201)
(857, 201)
(944, 256, 201)
(944, 201)
(947, 256, 201)
(947, 201)
(901, 256, 201)
(901, 201)
(539, 256, 201)
(539, 201)
(931, 256, 201)
(931, 201)
(657, 256, 201)
(657, 201)
(407, 256, 201)
(407, 201)
(809, 256, 201)
(809, 201)
(746, 256, 201)
(746, 201)
(697, 256, 201)
(697, 201)
(769, 256, 201)
(769, 201)
(789, 256, 201)
(789, 201)
(800, 256, 201)
(800, 201)
(628, 256, 201)
(628, 201)
(767, 256, 201)
(767, 201)


In [3]:
def calculate_welchs(data):
    welchs_by_epoch = np.empty((len(data),2,101))

    for i, epoch in enumerate(data):
        f,p = signal.welch(x=epoch, fs=250)
        welchs_by_epoch[i] = np.array([f,p])
    
    return welchs_by_epoch

In [5]:

welchs = []

for i_type, t in enumerate(['P', 'S']):
    for i in range(1,11):
        try:
            file = 'elec_prom_' + t + "%02d" % (i) + '.dmp'
            print(file)
            data = np.load(file)
            welchs.append(calculate_welchs(data)) 
            #print(data.shape)
            #print(repr(calculate_welchs(data)))
        except Exception as exception:
            print(str(exception))
            pass

elec_prom_P01.dmp


/home/matias/programas/anaconda3/lib/python3.6/site-packages/scipy/signal/spectral.py:1637: UserWarning: nperseg = 256 is greater than input length  = 201, using nperseg = 201
  .format(nperseg, input_length))


elec_prom_P02.dmp
elec_prom_P03.dmp
elec_prom_P04.dmp
elec_prom_P05.dmp
elec_prom_P06.dmp
elec_prom_P07.dmp
elec_prom_P08.dmp
elec_prom_P09.dmp
elec_prom_P10.dmp
elec_prom_S01.dmp
elec_prom_S02.dmp
elec_prom_S03.dmp
elec_prom_S04.dmp
elec_prom_S05.dmp
elec_prom_S06.dmp
elec_prom_S07.dmp
elec_prom_S08.dmp
elec_prom_S09.dmp
elec_prom_S10.dmp


In [6]:
def calculate_epochs_power_mean(welchs_by_epoch, frequency_range):
    freqs = welchs_by_epoch[0,0,:]
    freqs_idxs = np.where((freqs>=frequency_range[0]) * (freqs<frequency_range[1]))
    return welchs_by_epoch[:,1,freqs_idxs].mean(axis=2)



frequencies_ranges = {"delta": (0,4), "theta": (4,8), "alpha": (8,13), "beta": (13,30), "gamma": (30,125)}
# matriz [paciente,banda_freq,epochs,potencia]

potencias_mean_std = np.zeros((20,5,2))
for i in range(20):
    for freq_idx,frequency_range in enumerate(frequencies_ranges.values()):
        frequency_epochs_power_means = calculate_epochs_power_mean(welchs[i],frequency_range)
        potencias_mean_std[i,freq_idx,0] = frequency_epochs_power_means.mean()
        potencias_mean_std[i,freq_idx,1] = frequency_epochs_power_means.std()

        #frequency_power_means.dump(name + 'power_means.dmp')

In [7]:
column_names_mean = list(map(lambda x: x+"_mean",frequencies_ranges.keys()))
df_mean = pd.DataFrame(potencias_mean_std[:,:,0],columns=column_names_mean)
column_names_std = list(map(lambda x: x+"_std",frequencies_ranges.keys()))
df_std = pd.DataFrame(potencias_mean_std[:,:,1],columns=column_names_std)

labels = list("P"*10 + "S"*10)
df_labels = pd.DataFrame({"labels":labels})
pd.concat([df_mean,df_std,df_labels],axis=1)

,delta_mean,theta_mean,alpha_mean,beta_mean,gamma_mean,delta_std,theta_std,alpha_std,beta_std,gamma_std,labels
0,3.318997e-12,8.359852e-13,2.140406e-13,5.999927e-14,3.089191e-15,4.854060e-12,1.043152e-12,2.373796e-13,7.214694e-14,4.323068e-15,P
1,4.009078e-12,1.318657e-12,1.438555e-13,3.707107e-14,3.426323e-15,3.474209e-12,1.203138e-12,1.216249e-13,2.565711e-14,3.196974e-15,P
2,7.208167e-12,1.674200e-12,2.876949e-13,4.099559e-14,1.537359e-15,6.090011e-12,1.605617e-12,2.123532e-13,1.939445e-14,7.514494e-16,P
3,3.632586e-12,1.711795e-12,3.844794e-13,4.256765e-14,1.157852e-15,3.690426e-12,1.448283e-12,3.550259e-13,3.348758e-14,1.483740e-15,P
4,5.871341e-12,1.414307e-12,3.004096e-13,6.990782e-14,5.762502e-15,4.619075e-12,1.090295e-12,2.028861e-13,3.496279e-14,3.918855e-15,P
5,4.327036e-12,1.007841e-12,5.387930e-13,5.670316e-14,8.588728e-16,4.273751e-12,9.823795e-13,4.667667e-13,5.550046e-14,6.989552e-16,P
6,4.463560e-12,1.087597e-12,1.522770e-13,5.803440e-14,5.222616e-15,3.957058e-12,9.297277e-13,1.133733e-13,5.153577e-14,9.613897e-15,P
7,1.249794e-12,4.236199e-13,5.702112e-14,1.290863e-14,1.140742e-15,1.957607e-12,4.286518e-13,5.654276e-14,1.137890e-14,1.854504e-15,P
8,2.719834e-12,1.559148e-12,2.083246e-13,5.213623e-14,2.120851e-15,2.465026e-12,1.444146e-12,1.849338e-13,4.248410e-14,1.334136e-15,P
9,4.498473e-12,9.602589e-13,1.791209e-13,4.825006e-14,2.388773e-15,4.313276e-12,1.036142e-12,1.415723e-13,4.501895e-14,5.369507e-15,P
